# Pandeia for WFIRST Imaging

How to cite this code:

> Klaus M. Pontoppidan ; Timothy E. Pickering ; Victoria G. Laidler ; Karoline Gilbert ; Christopher D. Sontag, et al.
"Pandeia: a multi-mission exposure time calculator for JWST and WFIRST", Proc. SPIE 9910, Observatory Operations: Strategies, Processes, and Systems VI, 991016 (July 15, 2016); doi:10.1117/12.2231768; http://dx.doi.org/10.1117/12.2231768

This is an introductory notebook that provides an easy-to-use interface for making Pandeia ETC calculations. The Pandeia engine called by this notebook only supports WFIRST imaging and IFU (WFIRST grism calculations are not currently supported). This notebook has simplified some configuration options to present a summary of the steps most users will take to make WFIRST calculations. Users are encouraged to view <a href="https://jwst-docs.stsci.edu/display/JPP/JWST+Exposure+Time+Calculator+-+ETC">the JWST documentation</a> for an overview of the features and functionality of the Pandeia engine and an explanation of the quantities calculated by Pandeia.



Refer to the documentation links provided within the *Help* menu for general information on the Jupyter/IPython notebook interface and useful keyboard short-cuts.  The key things you need to know are that you must use ``Shift-Enter`` to execute a cell and that once a cell is executed, all data defined within it becomes available to all other cells. (You can also click the <i class="fa-step-forward fa"></i> icon in the toolbar to run a cell.)

This first cell sets up the imports and configuration that are required:  

In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'
from __future__ import print_function, division
import sys
import os
import numpy as np
import matplotlib
from matplotlib import style
#style.use('ggplot')  # see http://matplotlib.org/users/style_sheets.html
                     # for info on matplotlib styles
import json
matplotlib.rcParams['axes.grid'] = False
matplotlib.rcParams['image.origin'] = 'lower'
import matplotlib.pyplot as plt

# the first pandeia import is required to run calculations. the others are provided to
# allow setting up calculations and loading/saving of inputs or results.
from pandeia.engine.perform_calculation import perform_calculation
from pandeia.engine.calc_utils import get_telescope_config, get_instrument_config, build_default_calc, build_default_source
from pandeia.engine.io_utils import read_json, write_json

To run Pandeia, you must give it a calculation. Calculations are python dictionaries that specify every aspect of the detector configuration, instrument configuration, details of all of the sources in the scene, and observing strategies.

Let's break it down.

# First, we need to create a calculation. 

build_default_calc will create a default calculation for a WFIRST instrument, complete with a single faint point source.

build_default_calc() takes three arguments: telescope, instrument, mode.

Telescope is (obviously) wfirst.

Instrument is either wfirstimager or wfirstifu

The only mode available for wfirstimager is "imaging"; the only mode available for wfirstifu is "ifu".

For the WFIRST imager, run this cell:

In [2]:
calc = build_default_calc('wfirst','wfirstimager','imager')

or for the WFIRST IFU, run this cell:

In [ ]:
calc = build_default_calc('wfirst','wfirstifu','ifu')

This is a complete calculation, so we can run it. Try the following, and if it doesn't work, check your setup.

In [ ]:
results = perform_calculation(calc)

# Put together a scene of sources to observe

The first thing we need to do is customize the scene. The default that's been created for us is a single central point source with a flat spectral energy distribution, normalized to a flux of 0.001 mjy at 2.0 microns, with no redshift or extinction.

In [ ]:
calc['scene'][0]['spectrum']['normalization']['norm_flux'] = 0.001 # mjy
calc['scene'][0]['spectrum']['normalization']['norm_wave'] = 2.0 # microns
calc['scene'][0]['spectrum']['normalization']['type'] = "at_lambda"

The type of flux normalization can be changed. Run the cell below to normalize to a bandpass
- Valid bandpasses are:
   - bessel,J (or H or K)
   - cousins,I
   - johnson,V (or I, J, K)
   - sdss,u (or g, r, i, or z)
   - spitzer,irac3.6 (or irac4.5, irac5.8, irac8.0, mips24)
- You can also specify 'jwst' or 'hst' instead of 'photsys'; that requires specifying the instrument, mode, and filter though ('hst' and 'wfc3,ir,f140w', or 'jwst' and 'miri,imaging,f1280w'...)
   

In [ ]:
calc['scene'][0]['spectrum']['normalization']['norm_flux'] = 0.001 # mjy
calc['scene'][0]['spectrum']['normalization']['norm_wave'] = 2.0 # microns
calc['scene'][0]['spectrum']['normalization']['type'] = "photsys"
calc['scene'][0]['spectrum']['normalization']['bandpass'] = "sdss,z"

There are also other options for the SED, most notably a simple blackbody curve, or phoenix models, or one of the Brown galaxy spectra.

Run one of the following cells if you want to customize your source to a different SED type: 

In [ ]:
calc['scene'][0]['spectrum']['sed']['sed_type'] = 'blackbody' # shape
calc['scene'][0]['spectrum']['sed']['temp'] = 5770.0 # brightness temperature

In [ ]:
calc['scene'][0]['spectrum']['sed']['sed_type'] = 'phoenix' # shape
calc['scene'][0]['spectrum']['sed']['key'] = 'g2v' # spectral type

In [ ]:
calc['scene'][0]['spectrum']['sed']['sed_type'] = 'brown' # shape
calc['scene'][0]['spectrum']['sed']['key'] = 'ngc_3521' # galaxy profile

We can also change the shape of the source. Valid options are 'point', 'gaussian2d', 'sersic', and 'flat'. 

'point' (the default) is a point source, represented as a simple impulse function to convolve with the PSF.
'flat' is an ellipse step function, with a uniform distribution out to the bounding curve.
'gaussian2d' is a two-dimensional gaussian function, where major and minor are now the 1-sigma boundaries.
'sersic' is NOT the typical sersic formulation from Graham & Driver (1992); it is an e-folding version, defined as $e^{-r(\frac{1}{index})}$. index=4 is the standard De Vaucoulers profile, index 0.5 is similar to a gaussian, and index=1 is an exponential profile.

If you change the shape to gaussian2d, flat, or sersic, you have additional parameters to fill. In particular, there are options to change how you set the source brightness:
 - 'integ_infinity' Integrates the flux of the entire profile
 - 'surf_center' Normalizes to the surface brightness at the center of the profile
 - 'surf_scale' Normalizes to the surface brightness at the scale (e-folding for sersic, 1-sigma for gaussian2d) radius. You cannot choose this for the flat source.
If you choose surf_center or surf_scale, you need to specify whether the brightness is per square arcsecond ('arcsec^2') or steradians ('sr')

In [ ]:
calc['scene'][0]['shape']['geometry'] = 'point' # default

In [ ]:
calc['scene'][0]['shape']['geometry'] = 'gaussian2d'
calc['scene'][0]['shape']['major'] = 0.5 # arcsec
calc['scene'][0]['shape']['minor'] = 0.2 # arcsec
calc['scene'][0]['shape']['norm_method'] = 'integ_infinity'
calc['scene'][0]['shape']['surf_area_units'] = None

In [ ]:
calc['scene'][0]['shape']['geometry'] = 'flat'
calc['scene'][0]['shape']['major'] = 0.5 # arcsec
calc['scene'][0]['shape']['minor'] = 0.2 # arcsec
calc['scene'][0]['shape']['norm_method'] = 'surf_center'
calc['scene'][0]['shape']['surf_area_units'] = 'arcsec^2'

In [ ]:
calc['scene'][0]['shape']['geometry'] = 'sersic'
calc['scene'][0]['shape']['major'] = 0.5 # arcsec
calc['scene'][0]['shape']['minor'] = 0.2 # arcsec
calc['scene'][0]['shape']['sersic_index'] = 1.0
calc['scene'][0]['shape']['norm_method'] = 'surf_scale'
calc['scene'][0]['shape']['surf_area_units'] = 'sr'

We can also position the source: (though orientation obviously means little to a point source)

In [ ]:
calc['scene'][0]['position']['x_offset'] = 0.0 # arcsec
calc['scene'][0]['position']['y_offset'] = 0.0 # arcsec
calc['scene'][0]['position']['orientation'] = 45.0 # degrees

You can also add other sources, and customize them. The scene is a list of sources. The source we've been modifying is the zeroth source; anything you add will be calc['scene'][1] or higher when you customize it.

The two cells below will:

* Add a second source, configured as a point source
* Customize it to be a K0III object at 23rd magnitude through the WFC3 F098M filter, offset by 1 arcsecond to the east

In [ ]:
calc['scene'].append(build_default_source(geometry="point"))

In [ ]:
calc['scene'][1]['spectrum']['normalization']['norm_flux'] = 23
calc['scene'][1]['spectrum']['normalization']['norm_fluxunit'] = "vegamag"
calc['scene'][1]['spectrum']['normalization']['type'] = "hst"
calc['scene'][1]['spectrum']['normalization']['bandpass'] = "wfc3,ir,f098m"

calc['scene'][1]['spectrum']['sed']['sed_type'] = 'phoenix' # shape
calc['scene'][1]['spectrum']['sed']['key'] = 'k0iii' # spectral type

calc['scene'][1]['position']['x_offset'] = 1.0 # arcsec
calc['scene'][1]['position']['y_offset'] = 0.0 # arcsec
calc['scene'][1]['position']['orientation'] = 45.0 # degrees

# Now let's set up the instrument.

For the WFIRST Imager:

- Filters (For the WFIRST imager)
   - r062 (R-band 0.62 microns)
   - z087 (Z-band 0.87 microns) - the default
   - y106 (Y-band 1.06 microns)
   - j129 (J-band 1.29 microns)
   - w149 (w-band 1.49 microns)
   - h158 (H-band 1.58 microns)
   - f184 (f-band 1.84 microns)
   
There are no valid dispersive elements, and only one aperture: 'any'.

The detector can be configured to give multiple exposures in multiple groups and integrations, and set the readmode. (We are using the available JWST NIRCam configurations as stand-ins for the WFIRST detector, as the WFIRST WFI detectors are expected to be very similar to the NIRCam detectors.)

For the WFIRST imager, there are a number of readmodes:

- Readmodes:
   - 'rapid'
   - 'shallow2'
   - 'shallow4'
   - 'bright1'
   - 'bright2'
   - 'medium2'
   - 'medium8'
   - 'deep2' - the default
   - 'deep8'

- And a number of subarrays:
   - '32x32'
   - '64x64'
   - '128x128'
   - '256x256'
   - '512x512'
   - '1024x1024'
   - '1024x16'
   - 'full' (4096x4096) - the default

In [ ]:
calc['configuration']['instrument']['filter'] = "z087"
calc['configuration']['instrument']['aperture'] = "any"
calc['configuration']['instrument']['disperser'] = None
calc['configuration']['detector']['ngroup'] = 1 # groups per integration
calc['configuration']['detector']['nint'] = 10 # integrations per exposure
calc['configuration']['detector']['nexp'] = 1 # exposures
calc['configuration']['detector']['readmode'] = "deep2"
calc['configuration']['detector']['subarray'] = "full"

For the WFIRST IFU, there is much less to configure.
The only valid filter is "clear", to be used with the "prism" disperser in the "ifu" aperture, and "full" subarray. You can select between 'nrs' (the default) and 'nrsrapid' readmodes.

(We are using JWST NIRSpec settings as placeholders for the WFIRST IFU detector settings.)

In [ ]:
calc['configuration']['instrument']['filter'] = "clear"
calc['configuration']['instrument']['aperture'] = "ifu"
calc['configuration']['instrument']['disperser'] = "prism"
calc['configuration']['detector']['ngroup'] = 1 # groups per integration
calc['configuration']['detector']['nint'] = 10 # integrations per exposures
calc['configuration']['detector']['nexp'] = 1 # exposures
calc['configuration']['detector']['readmode'] = "nrs"
calc['configuration']['detector']['subarray'] = "full"

# We can now set up the observing strategy. 

For the WFIRST imager, there are a few options to set up the location, extraction aperture, and background extraction annulus through which the flux and SNR will be calculated.

In [ ]:
calc['strategy']['target_xy'] = [0.0,0.0] # x,y location to extract, in arcsec
calc['strategy']['aperture_size'] = 0.2 # radius of extraction aperture, in arcsec
calc['strategy']['sky_annulus'] = [0.4,0.6] # inner and outer radii of background subtraction annulus, in arcsec

For WFIRST IFU, there are three different observation strategies:
    - 'ifuapphot': IFU Aperture Photometry - the default
    - 'ifunodinscene': IFU Nod (in-scene)
    - 'ifunodoffscene': IFU Nod (off-scene)
    
For the IFU nod modes, you can set the dithers. For ifunodinscene, the values should be small- less than 5 arcsec. For ifunodinscene, it should be more than 10 arcsec. Note that we don't currently shift to add the dithers; this is a limitation of Pandeia as it currently stands. 

In [ ]:
calc['strategy']['method'] = 'ifunodinscene'
calc['strategy']['aperture_size'] = 0.2 # radius of extraction aperture, in arcsec
calc['strategy']['target_xy'] = [0.0,0.0] # x,y location to extract, in arcsec
calc['strategy']['dithers'] = [{'x':0.0,'y':0.0},{'x':1.0,'y':1.0}] # x,y offsets of dithers, in arcsec
calc['strategy']['reference_wavelength'] = None # Wavelength value (between 0.4 and 2.0 microns, non-inclusive) at which the 2D images and numerical outputs will be computed. If None, it chooses a default.

# Calculate!

We now have a formatted calculation we can run, so let's do that.

In [3]:
results = perform_calculation(calc)

Let's unpack the results.  Run the following to see the basic outputs - flux, SNR, exposure time, etc.

In [4]:
for x in results['scalar'].keys():
   print("{0:}:\t{1:}".format(x,results['scalar'][x]))

exposure_time:	1945.29
aperture_size:	0.2
cr_ramp_rate:	0.0473834147623
reference_wavelength:	0.859252643791
extraction_area:	10.3854302598
all_dithers_time:	1945.29
total_integrations:	1
saturation_time:	1934.66
measurement_time:	1913.4
sn:	63.4443546655
extracted_noise:	0.0767993279225
extracted_flux:	4.87248379879
duty_cycle:	0.983606557377
background:	0.140578399433
contamination:	0.0137624114687
background_sky:	2.59077521526
disperser:	None
filter:	z087
y_offset:	0.0
x_offset:	0.0
background_area:	51.927151299
background_total:	2.62692808041
total_exposure_time:	1945.29


Let's also look at some plots. Here are the detector SNR, signal, and saturation

In [ ]:
fig1 = plt.figure()
ax1 = fig1.add_subplot(131)
ax2 = fig1.add_subplot(132)
ax3 = fig1.add_subplot(133)
ax1.imshow(results['2d']['snr'])
ax2.imshow(results['2d']['detector'])
ax3.imshow(results['2d']['saturation'])
ax1.set_title('SNR')
ax2.set_title('Detector')
ax3.set_title('Saturation')
ax1.text(0.5,0.95,'Mean: {0:6.5f}'.format(np.mean(results['2d']['snr'])),horizontalalignment='center',transform=ax1.transAxes,color='#FFFFFF')
ax1.text(0.5,0.05,'Median: {0:6.5f}'.format(np.median(results['2d']['snr'])),horizontalalignment='center',transform=ax1.transAxes,color='#FFFFFF')
ax2.text(0.5,0.95,'Mean: {0:6.5f}'.format(np.mean(results['2d']['detector'])),horizontalalignment='center',transform=ax2.transAxes,color='#FFFFFF')
ax2.text(0.5,0.05,'Median: {0:6.5f}'.format(np.median(results['2d']['detector'])),horizontalalignment='center',transform=ax2.transAxes,color='#FFFFFF')
ax3.text(0.5,0.95,'Max: {0:}'.format(np.max(results['2d']['saturation'])),horizontalalignment='center',transform=ax3.transAxes,color='#FFFFFF')
ax3.text(0.5,0.05,'Min: {0:}'.format(np.min(results['2d']['saturation'])),horizontalalignment='center',transform=ax3.transAxes,color='#FFFFFF')
fig1.tight_layout()

We can also see some 1D plots. They won't look like much in Imager mode.

In [ ]:
fig2 = plt.figure()
ax1 = fig2.add_subplot(131)
ax2 = fig2.add_subplot(132)
ax3 = fig2.add_subplot(133)
if len(results['1d']['sn'][0]) == 1:
    ax1.scatter(results['1d']['sn'][0],results['1d']['sn'][1])
    ax2.scatter(results['1d']['extracted_flux_plus_bg'][0],results['1d']['extracted_flux_plus_bg'][1])
    ax3.scatter(results['1d']['extracted_flux'][0],results['1d']['extracted_flux'][1])
else:
    ax1.plot(results['1d']['sn'][0],results['1d']['sn'][1])
    ax2.plot(results['1d']['extracted_flux_plus_bg'][0],results['1d']['extracted_flux_plus_bg'][1])
    ax3.plot(results['1d']['extracted_flux'][0],results['1d']['extracted_flux'][1])
ax1.set_title('SNR')
ax2.set_title('Extracted Flux w/ Bkgd')
ax3.set_title('Extracted Flux w/o Bkgd')
fig2.tight_layout()

You can dump the calculation to a file (JSON format, which Python's json module can reconstruct into a dictionary) by running the following cell, and then copying the contents into a file.

In [5]:
print(json.dumps(calc, sort_keys=True, indent=4, separators=(',', ': ')))

{
    "background": "medium",
    "calculation": {
        "effects": {
            "background": true,
            "ipc": true,
            "saturation": true
        },
        "noise": {
            "crs": true,
            "darkcurrent": true,
            "ffnoise": true,
            "readnoise": true,
            "rn_correlation": true
        }
    },
    "configuration": {
        "detector": {
            "nexp": 1,
            "ngroup": 10,
            "nint": 1,
            "readmode": "deep2",
            "subarray": "full"
        },
        "dynamic_scene": true,
        "instrument": {
            "aperture": "any",
            "disperser": null,
            "filter": "z087",
            "instrument": "wfirstimager",
            "mode": "imaging"
        },
        "max_scene_size": 25,
        "scene_size": 5
    },
    "scene": [
        {
            "position": {
                "orientation": 0.0,
                "position_parameters": [
                    "x_offset

The saved file can be loaded back into a standalone Python instance (or Python script) and run through the engine.

For instance, if you saved your calculation to a file called input.json: